In [0]:
import json
json_string=""" {"ruleName":"Spacecraft Command Status Red","isSchedule":true,"Schedule":"1 Minute","scheduleTime":"1","scheduleTimePeriod":"Minute","filters":[{"topic_id":31,"trigger_operator":">","trigger_value":0.0,"filter":[{"name":"silverTimestamp","operator":"last","time":"1 Minute","filter_key_join_type":"AND"},{"name":"systemStatus","value":"RED","operator":"contains","filter_key_join_type":"AND"}],"group_key_join_type":"AND"}]}"""
data=json.loads(json_string)

In [0]:

     
import re
from datetime import datetime
from datetime import datetime, timedelta

def data_f():
  group_key=data['filters'][0]['group_key_join_type']
#   print(group_key)
  filters_length_l=data['filters']
  
  for pos_fil in range(0, len(filters_length_l)):
     first=data['filters'][pos_fil]
     # print(first)
    
    
     if 'trigger_operator' in first:
     #     print(first)
         trig_oper=data['filters'][pos_fil]['trigger_operator']
         trig_value=data['filters'][pos_fil]['trigger_value']
         filters_len=len(data['filters'][pos_fil]['filter'])
         topic_id=data['filters'][pos_fil]['topic_id']
         string_list1=[]
         string_list2=[]
         string_list3=[]
         string_list4=[]
         
 
         start=datetime.now()
        #  print(start)        
   
         for  pos in range(0,filters_len):
                 
              if 'time' in  data['filters'][pos_fil]['filter'][pos]:
                      
                         name=data['filters'][pos_fil]['filter'][pos]['name']
                         join_key=data['filters'][pos_fil]['filter'][pos]['filter_key_join_type']
                         time=data['filters'][pos_fil]['filter'][pos]['time']
                         str = re.findall(r'\d+', time)
                         time_value_str=""
                         time_value_str+="".join(str)
                         time_value=int(time_value_str)
                         operator=data['filters'][pos_fil]['filter'][pos]['operator']
                   
                         # if operator is'last'
                         if operator=='last':
                              if 'Minute' in time:
                                   end = start - timedelta(minutes=time_value)
                              elif 'Second' in time:
                                   end = start - timedelta(seconds=time_value)
                              else:
                                   end =start - timedelta(hours=time_value)
                              string=f"({name} between '{end}' AND '{start}')" 
                         # else operator is others'             
                         else:
                              if 'Minute' in time:
                                   end = start + timedelta(minutes=time_value)
                                
                              elif 'Second' in time:
                                   end = start + timedelta(seconds=time_value)
                              else:
                                   end =start +timedelta(hours=time_value)
                              string=f"({name} between '{start}' AND '{end}')"

                         #if filter is at end of the position
                         if filters_len-1==pos:
                            string_t=string
                         # filter is at other position    
                         else:
                             string_t=f"{string}{join_key}"
                         # append the string to the list 
                         # string_list1.append(string_t)
                         # store in variable with if  the loop   // 
                         string=string_t    
                                  
                             
                    
              else:     
                         name=data['filters'][pos_fil]['filter'][pos]['name']
                         value=data['filters'][pos_fil]['filter'][pos]['value']
                         operator=data['filters'][pos_fil]['filter'][pos]['operator']
                         join_key=data['filters'][pos_fil]['filter'][pos]['filter_key_join_type']
                         # if operator is'last'
                         if operator == 'contains':
                              operator=f'like %{value}%'
                         elif operator =='startsWith' :
                               operator=f'like {value}%'
                         elif  operator =='endsWith' :
                               operator=f'like %{value}'

                         # else operator is others'         
                         else:
                               operator=f"{operator} {value}"


                         if filters_len-1==pos:
                           string=f"({name} {operator})"           
                            
                         else:
                            string=f"({name} {operator}){join_key}"
              string_list1.append(string)
          #     print(string_list1) 
              result=""
              result+=" ".join(string_list1)     
                         #append the string to the list      
                         # string_list1.append(string)
                         ###// store it with else of loop //   
                         # string=string 
         result1=f"(select count(*) from {topic_id} where {result} ){trig_oper} {trig_value}"
        #  print(result1)   
         if   len(filters_length_l)-1 == pos_fil:
              #print("false")
              result1=f"{result1}"
         else:     
               #print("true")
               result1=f"{result1}{group_key}"
     #     print(result1)
         string_list3.append(result1)
         #print(string_list3)                 

                 
     else:
        aggreg_value=data['filters'][pos_fil]['aggregator_value']
        aggreg_oper=data['filters'][pos_fil]['aggregator_operator']
    
        name=data['filters'][pos_fil]['filter'][0]['name']
        aggregtr=data['filters'][pos_fil]['filter'][0]['aggregator']
        agg_name=data['filters'][pos_fil]['filter'][0]['aggregator_name']
        time=data['filters'][pos_fil]['filter'][0]['time']
        operator=data['filters'][pos_fil]['filter'][0]['operator']
        start=datetime.now()
        print(start)
        str = re.findall(r'\d+', time)
        time_value_str=""
        time_value_str+="".join(str)
        time_value=int(time_value_str)
        if operator =='last':
            
               if 'Minute' in time:
                                   end = start - timedelta(minutes=time_value)
                                   
               elif 'Second' in time:
                                   end = start - timedelta(seconds=time_value)
               else:
                                   end =start - timedelta(hours=time_value)
               string_t=f"{name} between '{end}' AND '{start}'"         
        else:
               if 'Minute' in time:
                                   end = start + timedelta(minutes=time_value)
                                   
               elif 'Second' in time:
                                   end = start + timedelta(seconds=time_value)
               else:
                                   end =start + timedelta(hours=time_value)
               string_t=f"{name} between '{start}' AND '{end}'"  
        string_list2.append(string_t)
     #    print(string_list2) 
        result=""
        result+=" ".join(string_list2) 
            

        result1=f"(select {aggregtr}({agg_name}) from {topic_id} where ( {result})){aggreg_oper}{aggreg_value}"
     #    print(result1)
        if   len(filters_length_l)-1 == pos_fil:
              print("false")
              result1=f"{result1}"
        else:  
               print("true")   
               result1=f"{result1}{group_key}"
     #    print(result1)

        string_list3.append(result1)
        #print(string_list3)

  result_f=""
  result_f+="".join(string_list3)
  result_fin=f"select case when({result_f}) then 'True' else 'False' END As ALERT"
  print(result_fin)             

       
    
                    
    
           
           
          

In [0]:
data_f()

select case when((select count(*) from 31 where (silverTimestamp between '2024-01-22 04:52:42.510606' AND '2024-01-22 04:53:42.510606')AND (systemStatus like %RED%) )> 0.0) then 'True' else 'False' END As ALERT
